# 1141_社會資料分析 作業九
學生：劉晏成

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：

1. 不同性別（a1）受訪者對於「是否減少搭乘汽機車」（c1b）的回答是否有顯著差異？
    - 以Cramer’s V 和 Lamda 檢定
2. 受訪者對於「與人結交，最好要小心一些」（f1a）與「 人與人之間再也沒有可靠而值得信任的關係。 」（trust_1）兩個問題的態度是否有顯著相關？
    - 以Gamma和 Tau-c係數檢定

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

In [14]:
variable_value_labels['c1b']

{1.0: '不曾',
 2.0: '很少',
 3.0: '通常',
 4.0: '總是',
 997.0: '拒答',
 998.0: '漏答',
 999.0: '不適用'}

## Q1: 不同性別（a1）受訪者對於「是否減少搭乘汽機車」（c1b）的回答是否有顯著差異？
    以Cramer’s V 和 Lamda 檢定

In [3]:
df_1 = df[["a1", "c1b"]].copy().rename(columns = { 'a1': 'sex', 'c1b': 'reduce_vehicle_willing' })

首先計算Cramer's V

In [4]:
from statistic.categorical_data import crosstab_with_residuals

report = crosstab_with_residuals(row_series = df_1['reduce_vehicle_willing'], col_series = df_1['sex'])

性別間的百分比

In [6]:
report['ct_col_pct']

sex,1.0,2.0
reduce_vehicle_willing,,
1.0,8.133971,4.444444
2.0,56.698565,46.000000
3.0,24.641148,29.333333
4.0,10.526316,20.222222


檢視性別內比較的調整後標準化殘差

In [11]:
report['adjusted_residuals']

sex,1.0,2.0
reduce_vehicle_willing,,
1.0,2.248619,-2.248619
2.0,3.150688,-3.150688
3.0,-1.55451,1.55451
4.0,-3.938436,3.938436


卡方檢定結果

In [7]:
report['chi2_table']

,Value
chi2,24.451853
p_value,0.000020
dof,3.000000


In [8]:
from statistic.categorical_data.cramers_v import cramers_v

cramers_v(row_series = df_1['sex'], col_series = df_1['reduce_vehicle_willing'])

{'cramers_v': 0.1678402134385773}

接下來實作Lambda Test。由於是否減少搭乘汽機車不影響個人性別，故採用以性別為自變數的不對稱Lambda 結果

In [9]:
from statistic.categorical_data.pre import goodman_kruskal_lambda, goodman_kruskal_tau

goodman_kruskal_lambda(df_1, 'sex', 'reduce_vehicle_willing')

,Measure,Type,Value,ASE,Approx_T,Approx_Sig
0,Lambda,Symmetric,0.052257,0.025789,2.026334,0.042731
1,Lambda,sex Dependent,0.105263,0.050499,2.084445,0.037120
2,Lambda,reduce_vehicle_willing Dependent,0.000000,0.000000,0.000000,1.000000


In [10]:
goodman_kruskal_tau(df_1, 'sex', 'reduce_vehicle_willing')

,Measure,Type,Value,ASE,Approx_T,Approx_Sig
0,Goodman-Kruskal Tau,sex Dependent,0.028170,0.010999,2.561109,0.010434
1,Goodman-Kruskal Tau,reduce_vehicle_willing Dependent,0.009567,0.004432,2.158746,0.030870


### Answer
由於「個人性別」基本上不被任何因素影響，故在此分析中，「個性」別為自變數，「是否減少搭乘汽機車」則為因變數。因此在Lambda 檢定中，可取以「個人性別」為自變數之數值，Appr. P Value 小於0.05 ，拒絕虛無假設，不同性別的「是否減少搭乘汽機車」狀況有顯著差異。

而卡方檢定的P Value 亦大於0.05 ，符合上述分析結論。

進一步分析交叉表以及調整後標準化殘差表。除了「通常」組別在兩性別中無達到顯著差異，其餘組別皆有顯著差異。其中顯示男性更多「不曾」、「很少」改變其習慣，而女性則顯著地「總是」改變其減少搭乘汽機車的習慣。

## Q2: 受訪者對於「與人結交，最好要小心一些」（f1a）與「 人與人之間再也沒有可靠而值得信任的關係。 」（trust_1）兩個問題的態度是否有顯著相關？
    以Gamma和 Tau-c係數檢定

use f21